<a href="https://colab.research.google.com/github/Praxis-QR/BDSN/blob/main/KK_C4_Spark_RDD_Dataframe_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

# Basic Data Analysis | Supermarket | Netflix | IMDB
Demonstrate the usage of RDD, Spark Dataframe and Spark SQL to explore the same datafile. <br>
More detailed tutorial available at https://github.com/Praxis-QR/BDSN/blob/main/Complex_Complete_Spark_Tutorial.ipynb
<hr>


In [1]:
from datetime import datetime
import pytz
print('ॐ श्री सरस्वत्यै नमः',datetime.now(pytz.timezone('Asia/Calcutta')))

!python --version
!lsb_release -a

#check which version of MongoDB  is available
#!apt-cache policy mongodb

#check which versions of software are available
#!pip3 index versions pyspark
#!pip3 index versions pyngrok

ॐ श्री सरस्वत्यै नमः 2024-01-19 19:21:04.048030+05:30
Python 3.10.12
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.3 LTS
Release:	22.04
Codename:	jammy


# Download Sample Data as CSV files
Sourced from https://github.com/search?q=owner%3Ajigsawlabs-student%20pyspark&type=repositories

In [2]:
import pandas as pd

#!wget -O netflix.csv "https://raw.githubusercontent.com/jigsawlabs-student/pyspark-cluster-lab/main/netflix_titles.csv"
!wget -q "https://github.com/Praxis-QR/BDSN/raw/main/Data2/NetFlix.csv"
#df = pd.read_csv('s3://jigsaw-labs-student/supermarket_sales.csv')
df1 = pd.read_csv('NetFlix.csv')

!wget -q 'https://github.com/Praxis-QR/BDSN/raw/main/Data2/SuperMarketSales.csv'
df2 = pd.read_csv('SuperMarketSales.csv')

#df2 = pd.read_csv("s3://jigsaw-labs-student/imdb_movies.csv")
!wget -q 'https://github.com/Praxis-QR/BDSN/raw/main/Data2/IMDBmovies.csv'
df3 = pd.read_csv('IMDBmovies.csv')
df1.shape, df2.shape, df3.shape

((7787, 12), (1000, 17), (2000, 7))

# Spark Installation


## Full Installation - Skip

In [ ]:
#!apt update > /dev/null
#!apt install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
# Get latest and correct version of Spark
#
# if the current version of Spark is not used, there may be errors
# check here for current versions http://apache.osuosl.org/spark
#
#!wget -q http://apache.osuosl.org/spark/spark-2.2.2/spark-2.2.2-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-2.4.3/spark-2.4.3-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
#!wget -q http://apache.osuosl.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz

#!tar xf spark-2.4.5-bin-hadoop2.7.tgz
#!tar xf spark-3.0.1-bin-hadoop3.2.tgz
#!tar xf spark-3.1.2-bin-hadoop3.2.tgz
#!pip install -q findspark
#!pip install -q pyspark

#import os
#os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"
#os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"
#os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"


In [ ]:
# findspark route is no more required
#import findspark
#findspark.init()
#
#findspark.init('/content/spark-2.2.2-bin-hadoop2.7')
#
# https://stackoverflow.com/questions/42223498/findspark-init-indexerror-list-index-out-of-range-error
#


## Direct Install with pip

In [3]:
!pip3 -q install pyspark
!pip3 -q install pyngrok
#!pip3 -q install s3fs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [4]:
from pyspark.sql.functions import col, to_date, to_timestamp
from pyspark.sql.functions import regexp_replace
#from pyspark.sql.types import StructType,StructField, StringType, DateType, TimestampType, IntegerType,DoubleType, FloatType,NumericType, DecimalType
from pyspark.sql.types import *
from pyspark.sql.functions import *
#
from pyspark.sql import SparkSession
#

In [5]:
# spark = SparkSession.builder.master("local[*]").getOrCreate()
# note UI port switched from default 4040 to 4050 to avoid clash with ngrok
#
spark39 = SparkSession.builder.master("local[*]").\
            config('spark.ui.port', '4050').\
            appName("Yantrajaal").\
            getOrCreate()
#
sc = spark39.sparkContext
#
# The spark Console UI is available in the link that will be displayed in this cell.
# If you do not wish to use the Console, you may skip the Tunnel part
sc

<SparkContext master=local[*] appName=Yantrajaal>

##Build Tunnel with ngrok <br>
This part required ONLY if you need to access Spark Console<br>
To get your ngrok token please visit https://dashboard.ngrok.com/login <br>
and create a free account

###Load ngrok token from GDrive<br>
this part may be skipped if ngrok can be placed directly in one of the two next steps

In [6]:
from google.colab import drive
drive.mount('/content/drive')
#
#!ls /content/drive/'My Drive'/Praxis/WebCredentials/
#
!cp /content/drive/'My Drive'/Praxis/WebCredentials/ngrokToken.py .

Mounted at /content/drive


In [7]:
# credential file for Prithwis Mukerjee
# this file needs to be uploaded into the VM

from ngrokToken import ngrokToken

#for the sake of privacy the following credentials need to be stored in a text file called ngrokToken.py
#in the Colab VM in the format given below
#

#otherwise, the values can be directly placed here

#ngrokToken = 'xxxxxxxxxxxxxxx'   # uncomment this line and place your own credentials here
#token is available at https://dashboard.ngrok.com/get-started/setup


#print(ngrokToken)
ngrokTokenCmd = 'ngrok authtoken '+ngrokToken

this code adapted from https://towardsdatascience.com/quickly-share-ml-webapps-from-google-colab-using-ngrok-for-free-ae899ca2661a

In [8]:
from pyngrok import ngrok

# you may place the token directly here
#!ngrok authtoken xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
get_ipython().system_raw(ngrokTokenCmd)

# Open a HTTP tunnel on the default port 80
#
public_url = ngrok.connect(4050)
#This is where the Spark Console UI will be visible
public_url

<NgrokTunnel: "https://ba7a-34-23-247-21.ngrok-free.app" -> "http://localhost:4050">

The next four cells are not required. Mainly used to check on the status of the Tunnel

In [9]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://ba7a-34-23-247-21.ngrok-free.app


In [ ]:
#!ps

In [ ]:
#!kill -9 1219

### S3 Access ( Optional )
This data is not used, kept for reference

In [10]:
!pip3 -q install s3fs           # error does not seem to matter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 23.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2023.6.0 requires fsspec==2023.6.0, but you have fsspec 2023.12.2 which is incompatible.


In [11]:
df = pd.read_csv('s3://jigsaw-labs-student/supermarket_sales.csv')
df.head(2)

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6


In [12]:
df = pd.read_csv("s3://jigsaw-labs-student/imdb_movies.csv")
df.head(2)

,title,genre,budget,runtime,year,month,revenue
0,Avatar,Action,237000000,162.0,2009,12,2787965087
1,Pirates of the Caribbean: At World's End,Adventure,300000000,169.0,2007,5,961000000


# Supermarket Sales Data

In [13]:
!head SuperMarketSales.csv

Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761904762,26.1415,9.1
226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.82,80.22,3/8/2019,10:29,Cash,76.4,4.761904762,3.82,9.6
631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761904762,16.2155,7.4
123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.288,489.048,1/27/2019,20:33,Ewallet,465.76,4.761904762,23.288,8.4
373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761904762,30.2085,5.3
699-14-3026,C,Naypyitaw,Normal,Male,Electronic accessories,85.39,7,29.8865,627.6165,3/25/2019,18:30,Ewallet,597.73,4.761904762,29.8865,4.1
355-53-5943,A,Yangon,Member,Female,Electronic accesso

## RDD Operations

### Read CSV into RDD

In [14]:
sales_RDD1 = sc.textFile("SuperMarketSales.csv")
sales_RDD2 = spark39.read.text("SuperMarketSales.csv").rdd
# for an explanation of the alternate way to read text files
# see this https://stackoverflow.com/questions/52665353/reading-a-text-file-in-spark/52669632#52669632

In [15]:
sales_RDD1.take(5)

['Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating',
 '750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761904762,26.1415,9.1',
 '226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.82,80.22,3/8/2019,10:29,Cash,76.4,4.761904762,3.82,9.6',
 '631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761904762,16.2155,7.4',
 '123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.288,489.048,1/27/2019,20:33,Ewallet,465.76,4.761904762,23.288,8.4']

In [16]:
sales_RDD2.take(5)

[Row(value='Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating'),
 Row(value='750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761904762,26.1415,9.1'),
 Row(value='226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.82,80.22,3/8/2019,10:29,Cash,76.4,4.761904762,3.82,9.6'),
 Row(value='631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761904762,16.2155,7.4'),
 Row(value='123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.288,489.048,1/27/2019,20:33,Ewallet,465.76,4.761904762,23.288,8.4')]

In [18]:
# Delete the first, header row
#
#!sed '1d' SuperMarketSales.csv > SuperMarketSales_NoHeader.csv
!sed -i '1d' SuperMarketSales.csv
sales_RDD1 = sc.textFile("SuperMarketSales.csv")
sales_RDD2 = spark39.read.text("SuperMarketSales.csv").rdd
sales_RDD1.take(5)
#sales_RDD2.take(5)

['750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761904762,26.1415,9.1',
 '226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.82,80.22,3/8/2019,10:29,Cash,76.4,4.761904762,3.82,9.6',
 '631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761904762,16.2155,7.4',
 '123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.288,489.048,1/27/2019,20:33,Ewallet,465.76,4.761904762,23.288,8.4',
 '373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761904762,30.2085,5.3']

In [19]:
sales_RDD2.take(5)

[Row(value='750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761904762,26.1415,9.1'),
 Row(value='226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.82,80.22,3/8/2019,10:29,Cash,76.4,4.761904762,3.82,9.6'),
 Row(value='631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761904762,16.2155,7.4'),
 Row(value='123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.288,489.048,1/27/2019,20:33,Ewallet,465.76,4.761904762,23.288,8.4'),
 Row(value='373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761904762,30.2085,5.3')]

### Data Analysis with RDD

In [20]:
# Total Number of Rows
sales_RDD1.count()

1000

#### Filter Data

In [21]:
# Filter, or segregate the data, based on payment method
#
sales_wallet = sales_RDD1.filter(lambda x: ",Ewallet," in x)
sales_cash = sales_RDD1.filter(lambda x: ",Cash," in x)
sales_card = sales_RDD1.filter(lambda x: ",Credit card," in x)
#
wallet_count = sales_wallet.count()
cash_count = sales_cash.count()
card_count = sales_card.count()
print(wallet_count, cash_count, card_count)

345 344 311


#### Set Operations

In [22]:
# Another way to segregate the data
#
sales_wallet = sales_RDD1.filter(lambda x: ",Ewallet," in x)
wallet_count = sales_wallet.count()
#
sales_non_wallet = sales_RDD1.subtract(sales_wallet)
non_wallet_count = sales_non_wallet.count()
print(wallet_count, non_wallet_count)

345 655


In [23]:
# Get Unique Values of field City
#
sales_RDD_csv = sales_RDD1.map(lambda x: x.split(","))
cities = sales_RDD_csv.map(lambda x: x[2]).distinct()
cities.collect()

['Naypyitaw', 'Yangon', 'Mandalay']

In [24]:
sales_RDD_csv = sales_RDD1.map(lambda x: x.split(","))
product_lines = sales_RDD_csv.map(lambda x: x[5]).distinct()
product_lines.collect()

['Health and beauty',
 'Home and lifestyle',
 'Electronic accessories',
 'Sports and travel',
 'Food and beverages',
 'Fashion accessories']

#### Aggregation
Calculating Count, Sum, Average etc

In [26]:
# Element 7 in the dataset is Quantity
# Convert str to int
#
def x4m1(text):
    elements = text.split(",")
    return(int(elements[7]))

In [27]:
# Extract Quantity from dataset
#
sales_wallet_Qty = sales_wallet.map(lambda x: x4m1(x))
sales_wallet_Qty.take(3)

[7, 8, 7]

In [28]:
# Calculate sum of all Quantity
#
wallet_total_Qty = sales_wallet_Qty.reduce(lambda x,y : x+y)
print('wallet total quantity = ',wallet_total_Qty, '\nwallet average quantity =', wallet_total_Qty/wallet_count)

wallet total quantity =  1892 
wallet average quantity = 5.484057971014493


#### Accumulators
Reduce the number of passes through the data

In [30]:
wallet_data = sales_wallet_Qty.aggregate(
    (0,0), # the initial value
    (lambda acc, value: (acc[0] + value, acc[1] + 1)), # combine value with acc
    (lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1])) # combine accumulators
    )

print('Wallet Quantity Total',wallet_data[0],\
      '\nWallet Quantity Count',wallet_data[1],\
      '\nWallet Quantity Average', wallet_data[0]/wallet_data[1])

Wallet Quantity Total 1892 
Wallet Quantity Count 345 
Wallet Quantity Average 5.484057971014493


#### Aggregation on Multiple (two) Data Elements

In [31]:
# Element 7 in the dataset is Quantity
# Element 15 is Gross Income
#
def x4m2(text):
    elements = text.split(",")
    return(int(elements[7]), float(elements[15]))

In [32]:
sales_wallet_Qty_Inc = sales_wallet.map(lambda x: x4m2(x))
sales_wallet_Qty_Inc.take(3)

[(7, 26.1415), (8, 23.288), (7, 30.2085)]

In [33]:
wallet_data = sales_wallet_Qty_Inc.aggregate(
    (0,0,0), # the initial value
    (lambda acc, value: (acc[0] + value[0], acc[1] + value[1], acc[2] + 1)), # combine value with acc
    (lambda acc1, acc2 : (acc1[0] + acc2[0], acc1[1] + acc2[1], acc1[2] + acc2[2])) # combine accumulators
    )

print('Wallet Quantity Total',wallet_data[0],\
      '\nWallet Income Total',wallet_data[1],\
      '\nWallet Count',wallet_data[2],\
      '\nWallet Quantity Average',wallet_data[0]/wallet_data[2],\
      '\nWallet Income Average', wallet_data[1]/wallet_data[2]\
      )
#print('Wallet Quantity Average = ', wallet_data[0]/wallet_data[2],'\nWallet Gross Income Average = ', wallet_data[1]/wallet_data[2])

Wallet Quantity Total 1892 
Wallet Income Total 5237.767 
Wallet Count 345 
Wallet Quantity Average 5.484057971014493 
Wallet Income Average 15.181933333333333


#### Chained Operations
Eliminate intermediate RDD creation

In [34]:
wallet_data = sc.textFile("SuperMarketSales_NoHeader.csv").\
    filter(lambda x: ",Ewallet," in x).\
    map(lambda x: x4m2(x)).\
    aggregate(                                                                      # Perform Aggregation on multiple fields
    (0,0,0),                                                                        # the initial value
    (lambda acc, value: (acc[0] + value[0], acc[1] + value[1], acc[2] + 1)),        # combine value with acc
    (lambda acc1, acc2 : (acc1[0] + acc2[0], acc1[1] + acc2[1], acc1[2] + acc2[2])) # combine accumulators
    )

print('Wallet Quantity Total',wallet_data[0],\
      '\nWallet Income Total',wallet_data[1],\
      '\nWallet Count',wallet_data[2],\
      '\nWallet Quantity Average',wallet_data[0]/wallet_data[2],\
      '\nWallet Income Average', wallet_data[1]/wallet_data[2]\
      )

#print(wallet_data[0],wallet_data[1],wallet_data[2])
#print('Wallet Quantity Average = ', wallet_data[0]/wallet_data[2],'\nWallet Gross Income Average = ', wallet_data[1]/wallet_data[2])

Wallet Quantity Total 1892 
Wallet Income Total 5237.767 
Wallet Count 345 
Wallet Quantity Average 5.484057971014493 
Wallet Income Average 15.181933333333333


### Convert RDD to Spark DataFrame

In [37]:
# Split string data on "," and convert string to relevant datatypes
#
#sales_RDD1_split1 = sales_RDD1.map(lambda l: l.split(","))
#sales_RDD1_split1.take(1)
#sales_RDD1_split2 = sales_RDD1_split1.map(lambda l: [l[0],l[1],l[2],l[3],l[4],l[5],float(l[6]),int(l[7]),float(l[8]),float(l[9]),l[10],l[11],l[12],float(l[13]),float(l[14]),float(l[15]),float(l[16])])
#sales_RDD1_split2.take(1)

def x4m3(text):
    l = text.split(",")
    return([l[0],l[1],l[2],l[3],l[4],l[5],float(l[6]),int(l[7]),float(l[8]),float(l[9]),l[10],l[11],l[12],float(l[13]),float(l[14]),float(l[15]),float(l[16])])

sales_RDD1_csv = sales_RDD1.map(lambda x: x4m3(x))
#sales_RDD1_split.take(1)

In [38]:
# Convert to dataframe
# Date, Time does not convert correctly
#
SalesDataSchema = StructType([
    StructField('Invoice ID', StringType(), True),
    StructField('Branch', StringType(), True),
    StructField('City', StringType(), True),
    StructField('Customer Type', StringType(), True),
    StructField('Gender', StringType(), True),
    StructField('Product Line', StringType(), True),
    StructField('Unit Price', FloatType(), True),
    StructField('Quantity', IntegerType(), True),
    StructField('Tax 5%', DoubleType(), True),
    StructField('Total', DoubleType(), True),
    StructField('Date', StringType(), True),
    StructField('Time', StringType(), True),
    StructField('Payment', StringType(), True),
    StructField('cogs', DoubleType(), True),
    StructField('gross margin percentage', DoubleType(), True),
    StructField('gross income', DoubleType(), True),
    StructField('Rating', DoubleType(), True)
])

#sales_df1 = spark39.createDataFrame(data = sales_RDD1_split2, schema = colSchema)
sales_df1 = spark39.createDataFrame(data = sales_RDD1_csv, schema = SalesDataSchema)

#sales_df1.printSchema()
#sales_df1.show(5)

In [39]:
# Fix Date, Time datatype
#
#sales_df1 = sales_df1.withColumn('Date',to_date(col('Date'), "M/d/y"))
#sales_df1 = sales_df1.withColumn('Time',to_timestamp(col('Time'), "HH:mm"))

sales_df1 = sales_df1.\
    withColumn('Date',to_date(col('Date'), "M/d/y")).\
    withColumn('Time',to_timestamp(col('Time'), "HH:mm"))

#sales_df1 = sales_df1.

sales_df1.printSchema()
sales_df1.show(5)

root
 |-- Invoice ID: string (nullable = true)
 |-- Branch: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Customer Type: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Product Line: string (nullable = true)
 |-- Unit Price: float (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Tax 5%: double (nullable = true)
 |-- Total: double (nullable = true)
 |-- Date: date (nullable = true)
 |-- Time: timestamp (nullable = true)
 |-- Payment: string (nullable = true)
 |-- cogs: double (nullable = true)
 |-- gross margin percentage: double (nullable = true)
 |-- gross income: double (nullable = true)
 |-- Rating: double (nullable = true)

+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+----------+-------------------+-----------+------+-----------------------+------------+------+
| Invoice ID|Branch|     City|Customer Type|Gender|        Product Line|Unit Price|Quantity| Tax 5%|   To

## Dataframe Operations

### Read CSV into Spark Dataframe

In [53]:
# Relaod Supermarket Sales Data WITH HEADERS
# Original File has had its header line deleted
#
!wget -O SuperMarketSales.csv -q 'https://github.com/Praxis-QR/BDSN/raw/main/Data2/SuperMarketSales.csv'

In [54]:
# Date is not inferred correctly
# Needs to be fixed in next step
#
sales_df2 = spark39.read.format("csv").option("header", "true").option("inferSchema", "true").load("./SuperMarketSales.csv")
#sales_df2.printSchema()
#sales_df2.show(5)

In [55]:
sales_df2 = sales_df2.withColumn('Date',to_date(col('Date'), "M/d/y"))
sales_df2.printSchema()
sales_df2.show(5)

root
 |-- Invoice ID: string (nullable = true)
 |-- Branch: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Customer type: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Product line: string (nullable = true)
 |-- Unit price: double (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Tax 5%: double (nullable = true)
 |-- Total: double (nullable = true)
 |-- Date: date (nullable = true)
 |-- Time: timestamp (nullable = true)
 |-- Payment: string (nullable = true)
 |-- cogs: double (nullable = true)
 |-- gross margin percentage: double (nullable = true)
 |-- gross income: double (nullable = true)
 |-- Rating: double (nullable = true)

+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+----------+-------------------+-----------+------+-----------------------+------------+------+
| Invoice ID|Branch|     City|Customer type|Gender|        Product line|Unit price|Quantity| Tax 5%|   T

### Display a subset of columns

In [56]:
# All three works
#
#sales_df2.select(sales_df2['Invoice ID'], sales_df2['gross income'],sales_df2['City'],sales_df2['Payment']).show(truncate=False)
#sales_df2.select(['Invoice ID','gross income','City','Payment']).show(truncate=False)
sales_df2['Invoice ID','gross income','City','Payment'].show(truncate=False)

+-----------+------------+---------+-----------+
|Invoice ID |gross income|City     |Payment    |
+-----------+------------+---------+-----------+
|750-67-8428|26.1415     |Yangon   |Ewallet    |
|226-31-3081|3.82        |Naypyitaw|Cash       |
|631-41-3108|16.2155     |Yangon   |Credit card|
|123-19-1176|23.288      |Yangon   |Ewallet    |
|373-73-7910|30.2085     |Yangon   |Ewallet    |
|699-14-3026|29.8865     |Naypyitaw|Ewallet    |
|355-53-5943|20.652      |Yangon   |Ewallet    |
|315-22-5665|36.78       |Naypyitaw|Ewallet    |
|665-32-9167|3.626       |Yangon   |Credit card|
|692-92-5582|8.226       |Mandalay |Credit card|
|351-62-0822|2.896       |Mandalay |Ewallet    |
|529-56-3974|5.102       |Mandalay |Cash       |
|365-64-0515|11.7375     |Yangon   |Ewallet    |
|252-56-2699|21.595      |Yangon   |Ewallet    |
|829-34-3910|35.69       |Yangon   |Cash       |
|299-46-1805|28.116      |Mandalay |Cash       |
|656-95-9349|24.1255     |Yangon   |Credit card|
|765-26-6951|21.783 

In [57]:
sales_df2['Invoice ID','gross income','City','Payment'].orderBy('City','gross income',ascending=False).show(truncate=False)

+-----------+------------+------+-----------+
|Invoice ID |gross income|City  |Payment    |
+-----------+------------+------+-----------+
|687-47-8271|49.49       |Yangon|Credit card|
|325-77-6186|45.325      |Yangon|Ewallet    |
|384-59-6655|44.397      |Yangon|Cash       |
|704-48-3927|44.335      |Yangon|Ewallet    |
|827-77-7633|44.1405     |Yangon|Cash       |
|139-32-4183|43.866      |Yangon|Ewallet    |
|805-86-0265|42.282      |Yangon|Cash       |
|698-98-5964|40.605      |Yangon|Credit card|
|138-17-5109|40.1445     |Yangon|Credit card|
|638-60-7125|39.824      |Yangon|Credit card|
|232-11-3025|39.385      |Yangon|Cash       |
|652-43-6591|38.916      |Yangon|Credit card|
|880-46-5796|38.46       |Yangon|Ewallet    |
|746-94-0204|37.458      |Yangon|Credit card|
|453-33-6436|37.248      |Yangon|Cash       |
|243-55-8457|37.22       |Yangon|Ewallet    |
|416-17-9926|37.11       |Yangon|Credit card|
|884-80-6021|36.735      |Yangon|Ewallet    |
|742-04-5161|36.39       |Yangon|C

### Display a subset of rows
"where" Condition in SQL

In [58]:
sales_df2['Invoice ID','gross income','City','Payment'][sales_df2['City'] == 'Yangon'].show(truncate=False)

+-----------+------------+------+-----------+
|Invoice ID |gross income|City  |Payment    |
+-----------+------------+------+-----------+
|750-67-8428|26.1415     |Yangon|Ewallet    |
|631-41-3108|16.2155     |Yangon|Credit card|
|123-19-1176|23.288      |Yangon|Ewallet    |
|373-73-7910|30.2085     |Yangon|Ewallet    |
|355-53-5943|20.652      |Yangon|Ewallet    |
|665-32-9167|3.626       |Yangon|Credit card|
|365-64-0515|11.7375     |Yangon|Ewallet    |
|252-56-2699|21.595      |Yangon|Ewallet    |
|829-34-3910|35.69       |Yangon|Cash       |
|656-95-9349|24.1255     |Yangon|Credit card|
|765-26-6951|21.783      |Yangon|Credit card|
|329-62-1586|8.2005      |Yangon|Credit card|
|636-48-8204|8.64        |Yangon|Ewallet    |
|549-59-1358|13.2945     |Yangon|Ewallet    |
|227-03-5010|21.036      |Yangon|Credit card|
|189-17-4241|8.767       |Yangon|Credit card|
|848-62-7243|11.2005     |Yangon|Cash       |
|595-11-5460|9.658       |Yangon|Credit card|
|129-29-8530|15.655      |Yangon|E

#### AND | OR

In [59]:
sales_df2['Invoice ID','gross income','City','Payment'][(sales_df2['City'] == 'Yangon') & (sales_df2['Payment'] == 'Ewallet')].\
    show(truncate=False)

+-----------+------------+------+-------+
|Invoice ID |gross income|City  |Payment|
+-----------+------------+------+-------+
|750-67-8428|26.1415     |Yangon|Ewallet|
|123-19-1176|23.288      |Yangon|Ewallet|
|373-73-7910|30.2085     |Yangon|Ewallet|
|355-53-5943|20.652      |Yangon|Ewallet|
|365-64-0515|11.7375     |Yangon|Ewallet|
|252-56-2699|21.595      |Yangon|Ewallet|
|636-48-8204|8.64        |Yangon|Ewallet|
|549-59-1358|13.2945     |Yangon|Ewallet|
|129-29-8530|15.655      |Yangon|Ewallet|
|272-65-1806|27.396      |Yangon|Ewallet|
|635-40-6220|35.84       |Yangon|Ewallet|
|287-21-9091|33.6015     |Yangon|Ewallet|
|873-51-0671|7.693       |Yangon|Ewallet|
|594-34-4444|4.858       |Yangon|Ewallet|
|865-92-6136|7.9125      |Yangon|Ewallet|
|704-48-3927|44.335      |Yangon|Ewallet|
|645-44-1170|26.1315     |Yangon|Ewallet|
|642-32-2990|5.48        |Yangon|Ewallet|
|575-30-8091|29.0        |Yangon|Ewallet|
|239-10-7476|23.385      |Yangon|Ewallet|
+-----------+------------+------+-

In [60]:
sales_df2['Invoice ID','gross income','City','Payment'][(sales_df2['City'] == 'Yangon') | (sales_df2['Payment'] == 'Ewallet')].\
    show(truncate=False)

+-----------+------------+---------+-----------+
|Invoice ID |gross income|City     |Payment    |
+-----------+------------+---------+-----------+
|750-67-8428|26.1415     |Yangon   |Ewallet    |
|631-41-3108|16.2155     |Yangon   |Credit card|
|123-19-1176|23.288      |Yangon   |Ewallet    |
|373-73-7910|30.2085     |Yangon   |Ewallet    |
|699-14-3026|29.8865     |Naypyitaw|Ewallet    |
|355-53-5943|20.652      |Yangon   |Ewallet    |
|315-22-5665|36.78       |Naypyitaw|Ewallet    |
|665-32-9167|3.626       |Yangon   |Credit card|
|351-62-0822|2.896       |Mandalay |Ewallet    |
|365-64-0515|11.7375     |Yangon   |Ewallet    |
|252-56-2699|21.595      |Yangon   |Ewallet    |
|829-34-3910|35.69       |Yangon   |Cash       |
|656-95-9349|24.1255     |Yangon   |Credit card|
|765-26-6951|21.783      |Yangon   |Credit card|
|329-62-1586|8.2005      |Yangon   |Credit card|
|319-50-3348|4.03        |Mandalay |Ewallet    |
|300-71-4605|21.51       |Naypyitaw|Ewallet    |
|371-85-5789|13.197 

### Aggregation | Group By

In [61]:
sales_df2.groupBy('Payment','City').count().show()

+-----------+---------+-----+
|    Payment|     City|count|
+-----------+---------+-----+
|Credit card| Mandalay|  109|
|Credit card|Naypyitaw|   98|
|       Cash|Naypyitaw|  124|
|       Cash|   Yangon|  110|
|    Ewallet| Mandalay|  113|
|Credit card|   Yangon|  104|
|    Ewallet|Naypyitaw|  106|
|       Cash| Mandalay|  110|
|    Ewallet|   Yangon|  126|
+-----------+---------+-----+



In [62]:
sales_df2['Payment','City','Quantity'].groupBy('Payment','City').sum().show()

+-----------+---------+-------------+
|    Payment|     City|sum(Quantity)|
+-----------+---------+-------------+
|Credit card| Mandalay|          599|
|Credit card|Naypyitaw|          543|
|       Cash|Naypyitaw|          696|
|       Cash|   Yangon|          572|
|    Ewallet| Mandalay|          593|
|Credit card|   Yangon|          580|
|    Ewallet|Naypyitaw|          592|
|       Cash| Mandalay|          628|
|    Ewallet|   Yangon|          707|
+-----------+---------+-------------+



In [63]:
sales_df2['Payment','Quantity','gross income'].groupBy('Payment').mean().show()

+-----------+-----------------+------------------+
|    Payment|    avg(Quantity)| avg(gross income)|
+-----------+-----------------+------------------+
|    Ewallet|5.484057971014493|15.181933333333335|
|       Cash|5.511627906976744|15.532470930232577|
|Credit card|5.536977491961415| 15.42904180064309|
+-----------+-----------------+------------------+



In [64]:
# This is REALLY CLEVER
#
groupBy51 = ['Payment']
aggregate51 = ['gross income','Quantity']
#funs = [mean, sum, max]
funs = [count,mean, sum]

exprs51 = [f(col(c)) for f in funs for c in aggregate51]

sales_df2['Payment','Quantity','gross income'].groupBy(groupBy51).agg(*exprs51).show()

+-----------+-------------------+---------------+------------------+-----------------+-----------------+-------------+
|    Payment|count(gross income)|count(Quantity)| avg(gross income)|    avg(Quantity)|sum(gross income)|sum(Quantity)|
+-----------+-------------------+---------------+------------------+-----------------+-----------------+-------------+
|    Ewallet|                345|            345|15.181933333333335|5.484057971014493|5237.767000000001|         1892|
|       Cash|                344|            344|15.532470930232577|5.511627906976744|5343.170000000006|         1896|
|Credit card|                311|            311| 15.42904180064309|5.536977491961415|4798.432000000001|         1722|
+-----------+-------------------+---------------+------------------+-----------------+-----------------+-------------+



## SQL Table Operations

### Dataframe to Temporary Table

In [65]:
sales_df2.createOrReplaceTempView("tSales")
spark39.sql("describe tSales").show()

+--------------------+---------+-------+
|            col_name|data_type|comment|
+--------------------+---------+-------+
|          Invoice ID|   string|   NULL|
|              Branch|   string|   NULL|
|                City|   string|   NULL|
|       Customer type|   string|   NULL|
|              Gender|   string|   NULL|
|        Product line|   string|   NULL|
|          Unit price|   double|   NULL|
|            Quantity|      int|   NULL|
|              Tax 5%|   double|   NULL|
|               Total|   double|   NULL|
|                Date|     date|   NULL|
|                Time|timestamp|   NULL|
|             Payment|   string|   NULL|
|                cogs|   double|   NULL|
|gross margin perc...|   double|   NULL|
|        gross income|   double|   NULL|
|              Rating|   double|   NULL|
+--------------------+---------+-------+



In [66]:
spark39.sql("show tables").show()
spark39.sql("select * from tsales limit 3").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|         |   tsales|       true|
+---------+---------+-----------+

+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+----------+-------------------+-----------+------+-----------------------+------------+------+
| Invoice ID|Branch|     City|Customer type|Gender|        Product line|Unit price|Quantity| Tax 5%|   Total|      Date|               Time|    Payment|  cogs|gross margin percentage|gross income|Rating|
+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+----------+-------------------+-----------+------+-----------------------+------------+------+
|750-67-8428|     A|   Yangon|       Member|Female|   Health and beauty|     74.69|       7|26.1415|548.9715|2019-01-05|2024-01-19 13:08:00|    Ewallet|522.83|            4.761904762|     26.1415|   9.1|
|226-31-3081|

### Temporary to Permanent Table

In [67]:
sql_drop_table = """drop table if exists praxisdb.sales_T"""

sql_drop_database = """drop database if exists praxisdb cascade"""

sql_create_database = """create database if not exists praxisdb location '/content/praxisdb/'"""

sql_create_table = """create table if not exists praxisdb.sales_T using parquet as select  * from tSales"""

spark39.sql(sql_drop_database)

spark39.sql(sql_create_database)

spark39.sql(sql_drop_table)

spark39.sql(sql_create_table)

DataFrame[]

In [68]:
spark39.sql("use praxisdb")
spark39.sql("show tables").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
| praxisdb|  sales_t|      false|
|         |   tsales|       true|
+---------+---------+-----------+



In [69]:
spark39.sql("select * from praxisdb.sales_t limit 5").show()

+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+----------+-------------------+-----------+------+-----------------------+------------+------+
| Invoice ID|Branch|     City|Customer type|Gender|        Product line|Unit price|Quantity| Tax 5%|   Total|      Date|               Time|    Payment|  cogs|gross margin percentage|gross income|Rating|
+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+----------+-------------------+-----------+------+-----------------------+------------+------+
|750-67-8428|     A|   Yangon|       Member|Female|   Health and beauty|     74.69|       7|26.1415|548.9715|2019-01-05|2024-01-19 13:08:00|    Ewallet|522.83|            4.761904762|     26.1415|   9.1|
|226-31-3081|     C|Naypyitaw|       Normal|Female|Electronic access...|     15.28|       5|   3.82|   80.22|2019-03-08|2024-01-19 10:29:00|       Cash|  76.4|            4.761904762| 

### Table Data Analysis

In [70]:
# NOTE backtick ` used to enclose column names with spaces inside!
spark39.sql("select payment, count(*), sum(quantity), avg(quantity), sum(`gross income`), avg(`gross income`) from praxisdb.sales_t group by payment ").show()

+-----------+--------+-------------+-----------------+-----------------+------------------+
|    payment|count(1)|sum(quantity)|    avg(quantity)|sum(gross income)| avg(gross income)|
+-----------+--------+-------------+-----------------+-----------------+------------------+
|    Ewallet|     345|         1892|5.484057971014493|5237.767000000001|15.181933333333335|
|       Cash|     344|         1896|5.511627906976744|5343.170000000006|15.532470930232577|
|Credit card|     311|         1722|5.536977491961415|4798.432000000001| 15.42904180064309|
+-----------+--------+-------------+-----------------+-----------------+------------------+



In [ ]:
#spark39.sql("select `gross income` from praxisdb.sales_t").show()

#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself.
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)